# W04 — Midpoint (Bash + cron + debug)

**Objetivo (60–90 min):** dejar DoD1 y DoD2 operativos (MVP) con evidencia enlazada. DoD3: 1 caso hoy.

### Checklist hoy
- [ ] `log_rotator.sh` + `backup.sh` en `./scripts/`
- [ ] `healthcheck.sh` en `./scripts/` y ejecutable
- [ ] `Makefile` con atajos (`make test-rotate`, `make backup`, `make health`)
- [ ] Enlace del repo pegado en *Evidence link* de DoD1 y DoD2
- [ ] DoD3: 1 caso resuelto (o hipótesis + comandos) y enlace


In [ ]:
# Escribe los scripts en ./scripts/
import os, stat
scripts_dir = "scripts"; os.makedirs(scripts_dir, exist_ok=True)
open(f"{scripts_dir}/log_rotator.sh","w").write('#!/usr/bin/env bash\nset -euo pipefail\n# Usage: ./log_rotator.sh "/path/to/*.log" KEEP_COUNT\nPATTERN="${1:-./tmp/logs/*.log}"\nKEEP="${2:-5}"\n\nrotate_file () {\n  local f="$1"\n  if [[ -f "$f" ]]; then\n    gzip -c "$f" > "${f}.$(date +%F).gz" || true\n    : > "$f"\n  fi\n  ls -1t "${f}."*.gz 2>/dev/null | tail -n +$((KEEP+1)) | xargs -r rm -f\n}\n\nfor f in $PATTERN; do\n  rotate_file "$f"\ndone\necho "[OK] rotation done for pattern=$PATTERN keep=$KEEP"\n')
open(f"{scripts_dir}/backup.sh","w").write('#!/usr/bin/env bash\nset -euo pipefail\n# Usage: ./backup.sh SRC_DIR DEST_DIR\nSRC="${1:-./}"\nDST="${2:-./backups}"\nmkdir -p "$DST"\ntar -czf "${DST}/backup-$(date +%F-%H%M).tar.gz" "$SRC"   --exclude=\'./backups\' --exclude=\'./tmp\' --exclude=\'./.git\' --exclude=\'./__pycache__\'\nfind "$DST" -type f -name "backup-*.tar.gz" -mtime +7 -delete\necho "[OK] backup created in $DST"\n')
open(f"{scripts_dir}/healthcheck.sh","w").write('#!/usr/bin/env bash\nset -euo pipefail\n\n# example checks (edit to your machine)\nCMD_OK="$(command -v docker || true)"\nPING_OK="$(ping -c1 8.8.8.8 >/dev/null 2>&1 && echo ok || echo fail)"\n\nFAILS=0\n\nif [[ -z "$CMD_OK" ]]; then\n  echo "docker: NOT FOUND"; ((FAILS++))\nelse\n  echo "docker: OK"\nfi\n\nif [[ "$PING_OK" != "ok" ]]; then\n  echo "network: FAIL"; ((FAILS++))\nelse\n  echo "network: OK"\nfi\n\n[[ $FAILS -gt 0 ]] && exit 1 || exit 0\n')
for n in ["log_rotator.sh","backup.sh","healthcheck.sh"]:
    p = f"{scripts_dir}/{n}"
    os.chmod(p, os.stat(p).st_mode | stat.S_IXUSR)
print("Scripts escritos en", scripts_dir)

In [ ]:
# Prepara entorno de prueba y ejecuta rotación de logs
import os, subprocess, pathlib, time
tmp_logs = pathlib.Path("tmp/logs"); tmp_logs.mkdir(parents=True, exist_ok=True)
log = tmp_logs/"app.log"
log.write_text("\n".join([f"{time.ctime()} sample line {i}" for i in range(200)]))
subprocess.run(["bash","scripts/log_rotator.sh", f"{tmp_logs}/*.log","3"], check=False)
print("gz files:", list(tmp_logs.glob("*.gz")))

In [ ]:
# Ejecuta backup (en ./backups) y healthcheck (puede fallar si no hay ping/docker)
import subprocess, os
os.makedirs("backups", exist_ok=True)
subprocess.run(["bash","scripts/backup.sh","." , "backups"], check=False)
subprocess.run(["bash","scripts/healthcheck.sh"], check=False)

### Cron (añádelo cuando lo adaptes a tu máquina)
```
*/15 * * * * /usr/local/bin/healthcheck.sh >> /var/log/healthcheck.log 2>&1
0 0 * * * /usr/local/bin/log_rotator.sh "/var/log/*.log" 5
```

### DoD3 — Debug drills (elige 1 hoy)
- **Disco lleno:** `du -sh`, `lsof | grep deleted` → limpia y documenta.
- **Proceso colgado:** `sleep 9999`; localiza con `ps/htop`; `kill -15` y explica señales.
- **Puerto ocupado:** `nc -l 8080`; detecta con `ss -lntp` y libera.
**Plantilla:** _síntoma → hipótesis → comandos → resultado → causa → solución_.
